# Processing Wind nudging data

### Set up
#### Packages

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from scipy import stats
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import timedelta
from cartopy.util import add_cyclic_point
from Processing_functions import FixLongitude, FixTime, CalcStatforDim, CalcStatbyGrpDim, Ensemble

#### Filepaths & name variables

In [2]:
## Test numbers
tst_nums = np.arange(1,11)
tst_type = 'nudge_ensemble'

## Test names
control = 'f.e22.F1850.f09_f09_mg17.control_test_nudge.'
rfn240K = 'f.e22.F1850.f09_f09_mg17.cri240K_test_nudge.'
rfn263K = 'f.e22.F1850.f09_f09_mg17.cri263K_test_nudge.'
rfn273K = 'f.e22.F1850.f09_f09_mg17.cri273K_test_nudge.'

## Time averaging type
time_avg = 3 # 0: Monthly, 1: Yearly, 2: Seasonal, 3: All data

## Ensemble mean or All members
ens_type = 'Mean'

## Filtering
filter = False
filter_str = 'filtered' if filter else 'non_filtered'

## Filepaths
path_to_arch = "/glade/derecho/scratch/glydia/archive/"
path_to_data = "/atm/hist/"
filename_ext = ".cam.h1"
path_to_outdata = '/glade/work/glydia/Arctic_CRI_processed_data/processed_wind_nudging_ensemble_data/'

## Variables to process
var_list = np.array(['FLDS','FLUT','T','TS','CLOUD','TGCLDLWP','LCC','U','V','Target_U','Target_V'])
var = var_list[0]

In [3]:
## Chunking variables
la_chunk = 64
lo_chunk = 96
le_chunk = 4

In [4]:
%%time
## Select plot type - yearly or monthly - to make and assign variables accordingly
# Monthly
if time_avg == 0:
    time_str = 'Month'
    grp_str = "time.month"
    tm_chunk = 30
    concat_str = 'month'

# Seasonal
elif time_avg == 2:
    time_str = 'Season'
    grp_str = "time.season"
    tm_chunk = 30
    concat_str = 'season'



# All-data average
elif time_avg == 3:
    time_str = 'All_data'
    tm_chunk = -1
    grp_str = ''

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs


In [5]:
%%time

## Set up ensemble index
ens_index = pd.Index(tst_nums, name="ensemble_member")

CPU times: user 117 µs, sys: 0 ns, total: 117 µs
Wall time: 119 µs


In [6]:
def LCC(ds):
    # Add LCC
    ds_lwp = ds['TGCLDLWP']

    da = xr.where(ds_lwp >= 0.005, 1, 0)
    da = da.rename('LCC')
    da.LCC.attrs['units'] = 'frequency'
    da.LCC.attrs['long_name'] = 'Liquid-containing cloud frequency'

    da.compute()
    return da

In [7]:
def LWP(ds):
    # Change units on LWP
    da = ds['TGCLDLWP']
    da *= 1000
    da.TGCLDLWP.attrs['units'] = 'g/m2'

    da.compute()
    return da

In [8]:
def filter_func(ds, da):
    ds_ts = ds.TS
    ds_lwp = ds.TGCLDLWP

    da = da.where(ds_ts <= 273)
    da = da.where(ds_lwp <= 0.09)
    da = da.where(ds_lwp >= 0.001)
    da.compute()
    return da

In [9]:
def LoadModEns(optics):
    ## Load data
    # Load ensemble members
    ds_list = []
    
    for i in tst_nums:
        print('ensemble member: '+str(i))
        
        # Open dataset
        tst_name = optics+str(i).zfill(3)
        path_i = path_to_arch+tst_name+path_to_data+tst_name+filename_ext+"*.nc"
        ds = xr.open_mfdataset(paths=path_i,chunks={'time':tm_chunk,'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})
    
        if var == 'LCC':
            # Add LCC
            dsv = LCC(ds)
    
        elif var == 'TGCLDLWP':
            # Change units on LWP
            dsv = LWP(ds)
    
        else:
            dsv = ds[var]

        dsv.compute()
    
        if filter:
            dsv = filter_func(ds, dsv)
            
        dsv = dsv.compute()
            
        dsv = FixLongitude(dsv)
    
        print('   made changes to ds')

        dsv.compute()

        ds_list.append(dsv)

    dsv = xr.concat(ds_list,ens_index)
    dsv.compute()

    # If doing ensemble mean
    if ens_type == 'Mean':
        ds_avg, ds_std, n_ds = CalcStatbyGrpDim(dsv, 'ensemble_member', grp_str, 'ensemble_member', 'time', 'ensemble_member')

        return ds_avg, ds_std, n_ds

    # If doing all ensemble members
    else:
        ds_avg, ds_std, n_ds = CalcStatforDim(dsv, grp_str, 'time') 

    return ds_avg, ds_std, n_ds

In [10]:
def SaveEns(data_avg, data_std, data_n, optics):
    # Compute changes to avg
    data_avg.compute()
    print('computed avg')
    
    data_avg.to_netcdf(path_to_outdata+optics+var+'.avg.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', format='NETCDF4')
    
    # Compute changes to avg
    data_std.compute()
    print('computed std')
    
    data_std.to_netcdf(path_to_outdata+optics+var+'.std.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', format='NETCDF4')
    
    # Compute changes to avg
    data_n.compute()
    print('computed n')
    
    data_n.to_netcdf(path_to_outdata+optics+var+'.n.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', format='NETCDF4')

### Load & modify data
#### Control data

In [11]:
%%time
ds_avg_control, ds_std_control, ds_n_control = LoadModEns(control)

print('processed all ensemble members')

ensemble member: 1
   made changes to ds
ensemble member: 2
   made changes to ds
ensemble member: 3
   made changes to ds
ensemble member: 4
   made changes to ds
ensemble member: 5
   made changes to ds
ensemble member: 6
   made changes to ds
ensemble member: 7
   made changes to ds
ensemble member: 8
   made changes to ds
ensemble member: 9
   made changes to ds
ensemble member: 10
   made changes to ds
processed all ensemble members
CPU times: user 9.65 s, sys: 12.5 s, total: 22.1 s
Wall time: 1min 26s


In [12]:
%%time

SaveEns(ds_avg_control, ds_std_control, ds_n_control, control)

computed avg
computed std
computed n
CPU times: user 6.06 ms, sys: 6.28 ms, total: 12.3 ms
Wall time: 72.2 ms


#### CRI240K data

In [13]:
%%time
ds_avg_cri240K, ds_std_cri240K, ds_n_cri240K = LoadModEns(rfn240K)

print('processed all ensemble members')

ensemble member: 1
   made changes to ds
ensemble member: 2
   made changes to ds
ensemble member: 3
   made changes to ds
ensemble member: 4
   made changes to ds
ensemble member: 5
   made changes to ds
ensemble member: 6
   made changes to ds
ensemble member: 7
   made changes to ds
ensemble member: 8
   made changes to ds
ensemble member: 9
   made changes to ds
ensemble member: 10
   made changes to ds
processed all ensemble members
CPU times: user 8.06 s, sys: 11 s, total: 19.1 s
Wall time: 1min 19s


In [14]:
%%time
SaveEns(ds_avg_cri240K, ds_std_cri240K, ds_n_cri240K, rfn240K)

computed avg
computed std
computed n
CPU times: user 11.1 ms, sys: 649 µs, total: 11.7 ms
Wall time: 69.4 ms


#### CRI263K data

In [15]:
%%time
ds_avg_cri263K, ds_std_cri263K, ds_n_cri263K = LoadModEns(rfn263K)

print('processed all ensemble members')

ensemble member: 1
   made changes to ds
ensemble member: 2
   made changes to ds
ensemble member: 3
   made changes to ds
ensemble member: 4
   made changes to ds
ensemble member: 5
   made changes to ds
ensemble member: 6
   made changes to ds
ensemble member: 7
   made changes to ds
ensemble member: 8
   made changes to ds
ensemble member: 9
   made changes to ds
ensemble member: 10
   made changes to ds
processed all ensemble members
CPU times: user 8.32 s, sys: 10.8 s, total: 19.2 s
Wall time: 1min 18s


In [16]:
%%time

SaveEns(ds_avg_cri263K, ds_std_cri263K, ds_n_cri263K, rfn263K)

computed avg
computed std
computed n
CPU times: user 7.25 ms, sys: 3.07 ms, total: 10.3 ms
Wall time: 30.9 ms


#### CRI273K data

In [17]:
%%time
ds_avg_cri273K, ds_std_cri273K, ds_n_cri273K = LoadModEns(rfn273K)

print('processed all ensemble members')

ensemble member: 1
   made changes to ds
ensemble member: 2
   made changes to ds
ensemble member: 3
   made changes to ds
ensemble member: 4
   made changes to ds
ensemble member: 5
   made changes to ds
ensemble member: 6
   made changes to ds
ensemble member: 7
   made changes to ds
ensemble member: 8
   made changes to ds
ensemble member: 9
   made changes to ds
ensemble member: 10
   made changes to ds
processed all ensemble members
CPU times: user 8.12 s, sys: 10.9 s, total: 19 s
Wall time: 1min 18s


In [18]:
%%time

SaveEns(ds_avg_cri273K, ds_std_cri273K, ds_n_cri273K, rfn273K)

computed avg
computed std
computed n
CPU times: user 10.5 ms, sys: 0 ns, total: 10.5 ms
Wall time: 65.5 ms
